<a href="https://colab.research.google.com/github/mearsdj/Amazon_Vine_Analysis/blob/main/NLP_tokenizer_and_stopwords_combo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [3]:
 # Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("StopWords").getOrCreate()

In [6]:
sentenceData = spark.createDataFrame([(
    0,["Big","data","is","super","powerful"]),
    (1,["This","is","going","to","be","epic"])
],["id","raw"])

sentenceData.show(truncate=False)

+---+--------------------------------+
|id |raw                             |
+---+--------------------------------+
|0  |[Big, data, is, super, powerful]|
|1  |[This, is, going, to, be, epic] |
+---+--------------------------------+



In [7]:
# Import stop words library
from pyspark.ml.feature import StopWordsRemover

# Run the Remover
remover = StopWordsRemover(inputCol="raw", outputCol="filtered")

remover.transform(sentenceData).show(truncate=False)

+---+--------------------------------+----------------------------+
|id |raw                             |filtered                    |
+---+--------------------------------+----------------------------+
|0  |[Big, data, is, super, powerful]|[Big, data, super, powerful]|
|1  |[This, is, going, to, be, epic] |[going, epic]               |
+---+--------------------------------+----------------------------+



In [8]:
#combined tokenizer and stop words remover!:
#create sentence df
dataframe = spark.createDataFrame([
    (0,"Spark is great"),
    (1,"We are learning spark"),
    (2,"Spark is better than hadoop, no doubt")],["id","sentence"])

from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol = "sentence",outputCol="words")

tokenized_df=tokenizer.transform(dataframe)
tokenized_df.show(truncate=False)


# Import stop words library
from pyspark.ml.feature import StopWordsRemover

# Run the Remover
remover = StopWordsRemover(inputCol="words", outputCol="filtered")

remover.transform(tokenized_df).show(truncate=False)

+---+-------------------------------------+---------------------------------------------+
|id |sentence                             |words                                        |
+---+-------------------------------------+---------------------------------------------+
|0  |Spark is great                       |[spark, is, great]                           |
|1  |We are learning spark                |[we, are, learning, spark]                   |
|2  |Spark is better than hadoop, no doubt|[spark, is, better, than, hadoop,, no, doubt]|
+---+-------------------------------------+---------------------------------------------+

+---+-------------------------------------+---------------------------------------------+-------------------------------+
|id |sentence                             |words                                        |filtered                       |
+---+-------------------------------------+---------------------------------------------+-------------------------------+
|0 